In [4]:
import json
import requests

def fetch_miners_data(block_heights):
    """
    Fetch miners data from the API.
    
    Args:
        block_heights (list): List of block heights to query
    
    Returns:
        dict: JSON response from the API
    """
    base_url = "http://5.78.102.130:8000/sigscore/miners/average-participation"
    
    # Convert block heights list to comma-separated string
    blocks_param = ",".join(map(str, block_heights))
    
    # Make the API request
    response = requests.get(f"{base_url}?blocks={blocks_param}")
    response.raise_for_status()  # Raise exception for bad status codes
    
    return response.json()

def validate_percentages(miners_data):
    """
    Validate that participation percentages sum to approximately 100%.
    
    Args:
        miners_data (dict): JSON data containing miner information
    
    Returns:
        bool: True if percentages are valid, False otherwise
    """
    total_percentage = sum(miner['avg_participation_percentage'] for miner in miners_data['miners'])
    if not (99.99 <= total_percentage <= 100.01):  # Allow for small floating-point errors
        print(f"Warning: Percentages sum to {total_percentage}%, not 100%")
        return False
    return True

def generate_distribution(miners_data, total_amount, token_name):
    """
    Generate a distribution JSON based on normalized participation percentages.
    """
    # First validate and print the original percentage sum
    total_percentage = sum(miner['avg_participation_percentage'] for miner in miners_data['miners'])
    print(f"Original percentage sum: {total_percentage}%")
    
    recipients = []
    running_total = 0
    
    # First pass: calculate amounts with normalized percentages
    for miner in miners_data['miners']:
        # Normalize the percentage
        normalized_percentage = (miner['avg_participation_percentage'] / total_percentage)
        amount = round(total_amount * normalized_percentage, 8)
        if amount > 0:
            recipients.append({
                "address": miner['miner_address'],
                "amount": amount
            })
            running_total += amount
    
    # Adjust for any rounding discrepancy
    if recipients:
        discrepancy = round(total_amount - running_total, 8)
        if discrepancy != 0:
            # Add the discrepancy to the largest recipient to maintain the total
            largest_recipient = max(recipients, key=lambda x: x['amount'])
            largest_recipient['amount'] = round(largest_recipient['amount'] + discrepancy, 8)
    
    # Verify final total
    final_total = sum(recipient['amount'] for recipient in recipients)
    print(f"Final distribution total: {final_total}")
    
    return {
        "distributions": [
            {
                "token_name": token_name,
                "recipients": recipients
            }
        ]
    }

def save_distribution_json(distribution, output_file):
    """
    Save distribution data to a JSON file.
    
    Args:
        distribution (dict): Distribution data to save
        output_file (str): Path to the output JSON file
    """
    with open(output_file, 'w') as f:
        json.dump(distribution, f, indent=2)



In [6]:
def main():
    # Example block heights from your query
    block_heights = [1439523, 1439837, 1440058, 1440237, 1441003, 1441626, 1442364, 1442507, 1443013, 1443371, 1444365]
    
    try:
        # Fetch data from API
        miners_data = fetch_miners_data(block_heights)
        
        # Generate distribution with example values
        total_amount = 4.9  # Example amount
        token_name = "ERGO"  # Example token name
        
        result = generate_distribution(miners_data, total_amount, token_name)
        
        # Pretty print the result to console
        print(json.dumps(result, indent=2))
        
        # Save to JSON file
        output_file = "distribution_output.json"
        save_distribution_json(result, output_file)
        print(f"\nDistribution data saved to {output_file}")
        
    except requests.RequestException as e:
        print(f"Error fetching data from API: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

if __name__ == "__main__":
    main()

Original percentage sum: 103.37350000000005%
Final distribution total: 4.899999999999999
{
  "distributions": [
    {
      "token_name": "ERGO",
      "recipients": [
        {
          "address": "9f6PPoCYrvHQbS6U8s7y6wE1S6pFET712ajBCzH5Ep7mYTGBudy",
          "amount": 1.1178135
        },
        {
          "address": "9eoPRhRmJQ71TPSkk2eUmMXYpxUVvZvFQo5uB9qjPBM4e3hC82J",
          "amount": 0.63323376
        },
        {
          "address": "9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2TuYsAik",
          "amount": 0.31606466
        },
        {
          "address": "9fUBHUXfduFkCvHcx1eMV799a8uUp3wxYh4WkNNLwhLtURmPMWh",
          "amount": 0.2694174
        },
        {
          "address": "9i4DWcdw6c4PMdSHvBRJs2aPQ5rqzd4R4Vf1JPxeVi7wsZVd4c1",
          "amount": 0.15812002
        },
        {
          "address": "9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsNLUuxo",
          "amount": 0.14425999
        },
        {
          "address": "9h6bjuzth7XDNeBxwFjpYSfmUkfBiepE8M

In [7]:
import requests
import csv
from datetime import datetime

# Define the API endpoint URL
api_url = "http://5.78.102.130:8000/sigscore/miners/bonus"

# Function to fetch data from API
def fetch_data_from_api(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Will raise an exception for bad status codes
        return response.json()
    except requests.RequestException as e:
        print(f"An error occurred while fetching data: {e}")
        return None

# Function to convert JSON to CSV
def json_to_csv(data, output_file):
    if not data:
        print("No data to process.")
        return
    
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['address', 'days_active', 'weekly_avg_hashrate', 'current_balance', 'last_payment']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for entry in data:
            # Convert datetime string to a more readable format if needed
            entry['last_payment'] = datetime.strptime(entry['last_payment'], "%Y-%m-%dT%H:%M:%S.%f%z").strftime("%Y-%m-%d %H:%M:%S")
            writer.writerow(entry)

# Main execution
if __name__ == "__main__":
    data = fetch_data_from_api(api_url)
    if data:
        json_to_csv(data, 'output.csv')
        print("Data has been successfully written to 'output.csv'.")
    else:
        print("Failed to fetch or process data.")

TypeError: strptime() argument 1 must be str, not None

In [2]:
import csv

# Define the addresses you want to remove
addresses_to_remove = [
    "9hzkS2hfEjv8yzhhTWmuYYxCXPK4e1kxYDxXMc5whZ57SJUoZyH",
    "9fs1QU7tsDvYNo1aVmMManemdF5Ud2QLavCTFABP3Smm3HrwX6B",
    "9fq3byFPSNBZogH7R4k8cLmeFiYghS6whfdMWcumctaAsrtnK2m",
    "9feFmoH5ftSpUVZXL9NATBjw5G3JHcBqbA3vEEcE7auHBvN7fAA",
    "9g5jXf74UWyXEJ4BZJ8gVp2ZJzG9bgPVijCtuUrWvv5DWHntE16",
    "9haAysnicciRTb8Qexr3Qogrv1HwDk7AUKvoQkeSFnNMj3fSVDK",
    # Add more addresses if needed
]

# Function to remove specified addresses from CSV
def remove_addresses(input_file, output_file, addresses):
    with open(input_file, 'r', newline='') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            if row['address'] not in addresses:
                writer.writerow(row)
            else:
                print(row['address'])

# Main execution
if __name__ == "__main__":
    input_csv = 'output.csv'  # The CSV file generated by the previous script
    output_csv = 'filtered_output.csv'  # New CSV file without the specified addresses
    
    remove_addresses(input_csv, output_csv, addresses_to_remove)
    print(f"Addresses removed. Updated CSV saved as '{output_csv}'.")

Addresses removed. Updated CSV saved as 'filtered_output.csv'.
